In [ ]:
import sys
import os
import boto3
import rasterio

import numpy as np

from rasterio.merge import merge

# Import raster helpers
sys.path.insert(0, "../../../../GOSTrocks/src")

import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

# Import GOST urban functions
sys.path.insert(1, "../../../src")

# Import local functions
# from novelUrbanization import *

%load_ext autoreload
%autoreload 2

from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

s3client = boto3.client("s3", verify=False)
out_folder = r"C:\WBG\Work\Projects\Novel_Urban\Mosaicks"

## Mosaick the selected population layers

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/"
pop_layer = "_cpo15.tif"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
pop_tiffs = []

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if (
            ("FINAL_STANDARD" in res["Key"])
            and (  # Look for the 250m data
                res["Key"].endswith(pop_layer)
            )
            and ("1k" not in res["Key"])
            and ("NO_DATA" not in res["Key"])
        ):
            pop_tiffs.append(f's3://{bucket}/{res["Key"]}')

In [ ]:
out_file = os.path.join(out_folder, f"merged_{pop_layer}")
merged_data, merged_profile = rMisc.merge_rasters(
    pop_tiffs, gdal_unssafe=True, compress=True
)

In [ ]:
dtype = "uint16"
merged_data[merged_data < 0] = 0
merged_data = merged_data.astype(dtype)
merged_profile.update(
    dtype=dtype,
    nodata=0,
)

with rasterio.open(out_file, "w", **merged_profile) as dst:
    dst.write(merged_data)

# Mosaic the DoU layers

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
urban_tiff = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if (
            ("FINAL_STANDARD" in res["Key"])
            and (
                res["Key"].endswith("urban.tif") or res["Key"].endswith("urban_hd.tif")
            )
            and ("1k" not in res["Key"])
        ):
            cur_pop = os.path.basename(res["Key"]).split("_")[1]
            cur_type = os.path.basename(res["Key"]).split("_")[-1].replace(".tif", "")
            cur_lyr = f"{cur_pop}_{cur_type}"
            cur_path = f"s3://{bucket}/{res['Key']}"
            try:
                urban_tiff[cur_lyr].append(cur_path)
            except:
                urban_tiff[cur_lyr] = [cur_path]

In [ ]:
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in urban_tiff.items():
    if "cpo15" in cur_lbl:
        tPrint(cur_lbl)
        out_file = os.path.join(out_folder, f"{cur_lbl}_DoU_Africa_mosaic.tif")
        curD, profile = rMisc.merge_rasters(
            rasters,
            merge_method="max",
            boolean_gt_0=True,
            out_file=out_file,
            gdal_unssafe=True,
            compress=True,
        )

In [ ]:
opened_tiffs = [rasterio.open(x) for x in hd_urban_tiff]
merged, out_transform = merge(opened_tiffs, method="max")
merged = ((merged > 0) * 1).astype("uint8")

# Create a new raster file with the merged data
metadata = opened_tiffs[0].meta.copy()
metadata.update(
    {
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_transform,
        "dtype": "uint8",
    }
)
with rasterio.open(
    f"/home/wb411133/temp/{sel_pop}_combo_urban_hd.tif", "w", **metadata
) as dst:
    dst.write(merged)

# Mosaick data from PP

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/AAPPC/Delineations/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
db_tiffs = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if "1k" not in res["Key"]:
            cur_path = f"s3://{bucket}/{res['Key']}"
            cur_type = os.path.basename(cur_path).split("_")[2][:2]
            if cur_type in ["cc", "co", "ur", "ce"]:
                cur_pop = os.path.basename(cur_path).split("_")[1].split("d")[0]
                cur_label = f"{cur_pop}_{cur_type}"
                try:
                    db_tiffs[cur_label].append(cur_path)
                except:
                    db_tiffs[cur_label] = [cur_path]

In [ ]:
db_tiffs.keys()

In [ ]:
cur_lbl = "cpo15_ce"
out_file = os.path.join(out_folder, f"{cur_lbl}_DB_Africa_mosaic_compress.tif")
curD, profile = rMisc.merge_rasters(
    db_tiffs[cur_lbl],
    merge_method="max",
    boolean_gt_0=True,
    out_file=out_file,
    gdal_unssafe=True,
    compress=True,
)

In [ ]:
profile

In [ ]:
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in db_tiffs.items():
    tPrint(cur_lbl)
    out_file = os.path.join(out_folder, f"{cur_lbl}_DB_Africa_mosaic.tif")
    if not os.path.exists(out_file):
        curD, profile = rMisc.merge_rasters(
            rasters,
            merge_method="max",
            boolean_gt_0=True,
            out_file=out_file,
            gdal_unssafe=True,
            compress=True,
        )

### Combine PP layers into single raster

In [ ]:
try:
    del final
except:
    pass

in_rasters = []
for lbl, val in [
    ["ur", 1],
    ["cc", 2],
    ["co", 3],
    ["ce", 4],
]:
    in_raster = os.path.join(out_folder, f"cpo15_{lbl}_DB_Africa_mosaic.tif")
    new_data = rasterio.open(in_raster).read(1) * val
    in_rasters.append(new_data)

In [ ]:
combo_urban = np.stack(in_rasters)
max_urban = np.max(combo_urban, axis=0)
max_urban.shape

In [ ]:
inR = rasterio.open(in_raster)
with rasterio.open(
    os.path.join(out_folder, "cpo15_merged_PP_DB_Africa_mosaic.tif"), "w", **inR.profile
) as dst:
    dst.write(max_urban, 1)

In [ ]:
out_folder